To run predictions, you first need to place the trained model file (.pt format) into the current directory

In [ ]:
import chemprop
import pandas as pd
from sklearn.metrics import mean_absolute_error, mean_squared_error
from sklearn.decomposition import PCA
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error
import numpy as np

In [ ]:
df = pd.read_csv('data/separate_data/SGNN_dataset.csv')
train = df[df['dataset_type'] == 'train']
train = train.drop(columns=['dataset_type'])
val = df[df['dataset_type'] == 'validation']
val= val.drop(columns=['dataset_type'])
test = df[df['dataset_type'] == 'test']
test = test.drop(columns=['dataset_type'])
train.to_csv('train_data.csv', index=False)
val.to_csv('val_data.csv', index=False)
test.to_csv('test_data.csv', index=False)

In [3]:
df = pd.read_csv(f'test_data.csv')

In [4]:
seed = 42

In [5]:
checkpoint_dir = f"test_checkpoints_reg_seed_{seed}"
checkpoint_path = f"{checkpoint_dir}/model.pt"
save_dir = f"test_checkpoints_reg_seed_{seed}"  
model_objects = chemprop.utils.load_checkpoint(checkpoint_path)

/home/slavick/miniconda3/envs/logp_prediction_env/lib/python3.8/site-packages/chemprop/utils.py:107: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  state = torch.load(path, m

Loading pretrained parameter "encoder.encoder.0.cached_zero_vector".
Loading pretrained parameter "encoder.encoder.0.W_i.weight".
Loading pretrained parameter "encoder.encoder.0.W_h.weight".
Loading pretrained parameter "encoder.encoder.0.W_o.weight".
Loading pretrained parameter "encoder.encoder.0.W_o.bias".
Loading pretrained parameter "ffn.1.weight".
Loading pretrained parameter "ffn.1.bias".
Loading pretrained parameter "ffn.4.weight".
Loading pretrained parameter "ffn.4.bias".
Moving model to cuda


In [6]:
arguments = [
    '--test_path', 'test_data.csv',
    '--preds_path', 'test_preds_reg.csv',
    '--checkpoint_dir', save_dir
]

args = chemprop.args.PredictArgs().parse_args(arguments)
preds = chemprop.train.make_predictions(args=args)

/home/slavick/miniconda3/envs/logp_prediction_env/lib/python3.8/site-packages/chemprop/utils.py:310: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  args.from_dict(vars(torch.

Loading training args
Setting molecule featurization parameters to default.
Loading data


2053it [00:00, 15166.00it/s]
100%|██████████| 2053/2053 [00:00<00:00, 310202.32it/s]


Validating SMILES
Test size = 2,053
Predicting with an ensemble of 1 models


  0%|          | 0/1 [00:00<?, ?it/s]/home/slavick/miniconda3/envs/logp_prediction_env/lib/python3.8/site-packages/chemprop/utils.py:277: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental 

Loading pretrained parameter "encoder.encoder.0.cached_zero_vector".
Loading pretrained parameter "encoder.encoder.0.W_i.weight".
Loading pretrained parameter "encoder.encoder.0.W_h.weight".
Loading pretrained parameter "encoder.encoder.0.W_o.weight".
Loading pretrained parameter "encoder.encoder.0.W_o.bias".
Loading pretrained parameter "ffn.1.weight".
Loading pretrained parameter "ffn.1.bias".
Loading pretrained parameter "ffn.4.weight".
Loading pretrained parameter "ffn.4.bias".
Moving model to cuda


100%|██████████| 1/1 [00:01<00:00,  1.51s/it]

Saving predictions to test_preds_reg.csv
Elapsed time = 0:00:02


In [7]:
df[f'logP_pred_{seed}'] = [x[0] for x in preds]
df

,molecule,logp,logP_pred_42
0,c1cc2ccc3ccc4ccc5ccc6ccc1c1c2c3c4c5c61,7.64,6.612525
1,CCC1(CC)C(=O)NC(=S)NC1=O,1.50,1.333640
2,COC(=O)C(C#N)=NNc1ccc(Cl)c(C(F)F)c1,3.80,4.372901
3,CC(=O)C=Cc1ccccc1,2.07,2.203551
4,Cc1ccc2cccc(O)c2n1,2.33,2.202385
...,...,...,...
2048,N#CNc1ccccc1,1.87,1.300820
2049,NS(=O)(=O)c1ccc(S(=O)(=O)CCO)s1,-0.54,-0.666760
2050,CCCCC(NC(Cc1ccccc1)C(=O)N1CCC(OCOC)CC1)C(=O)NC...,3.50,3.591919
2051,O=C(COc1ccc2ccccc2c1)N1CCN(S(=O)(=O)c2ccc3c(c2...,4.47,4.520264


In [8]:
r2 = r2_score(df['logp'], df[f'logP_pred_{seed}'])
rmse = np.sqrt(mean_squared_error(df['logp'], df[f'logP_pred_{seed}']))
mae = mean_absolute_error(df['logp'], df[f'logP_pred_{seed}'])
round(rmse, 3), round(r2, 3), round(mae, 3)

(0.493, 0.931, 0.305)

In [ ]:
df.to_csv('Seeds_D_MMPN_PREDICT_SGNN_train_SGNN_test.csv', index=False)